# Preparing the Diataxis data for fine-tuning

In [11]:
import re
from fastcore.foundation import L
from fastcore.basics import Path
from datasets import Dataset
import pandas as pd

data_path = "/Users/cck/projects/chaski-llm/data/diataxis/raw_outputs/all_raw.txt"
data_path = Path(data_path)
raw_data = L(data_path.read_text().split("\n"))

In [13]:
# The regular expression pattern
pattern = r'^[QA]\d+$'

def is_qa_line(line):
    "Checks if we are dealing with a QA line in the Diataxis data"
    if line in ("", " ", "\n"):
        return False
    contents = line.split(":")
    if len(contents) == 1:
        return False
    before, *_ = contents
    if re.match(pattern, before):
        return True
    else:
        return False
    
data = raw_data.filter(is_qa_line)

def strictly_sanitize_string(input_string):
    # This pattern matches anything that's not a Q or A
    pattern = r'[^QA]'
    # Replace matched characters with an empty string
    strictly_sanitized_string = re.sub(pattern, '', input_string)
    return strictly_sanitized_string

def clean_qa_line(line):
    "Cleans a QA line in the Diataxis data"
    # Remove the question and answer labels
    contents = line.split(":")
    before = contents[0]
    before = strictly_sanitize_string(before).strip()
    return f"{before}:{':'.join(contents[1:])}"

data = data.map(clean_qa_line)

In [14]:
# split off the questions and pairs, which are next to each other
questions = data[0::2]
pairs = data[1::2]

In [15]:
questions[0]

'Q: How can the application of Diátaxis be described for most documentation projects, and what factors contribute to its straightforward implementation?'

In [16]:
pairs[0]

'A: The application of Diátaxis to most documentation projects can be described as fairly straightforward, largely due to the clear boundaries of the product defining the domain of concern and the ease of arranging contents according to the principles of tutorials, how-to guides, reference, and explanation.'

In [20]:
questions[:2], pairs[:2]

((#2) ['Q: How can the application of Diátaxis be described for most documentation projects, and what factors contribute to its straightforward implementation?','Q: What role do landing pages play in the basic structure of documentation organized according to Diátaxis principles?'],
 (#2) ['A: The application of Diátaxis to most documentation projects can be described as fairly straightforward, largely due to the clear boundaries of the product defining the domain of concern and the ease of arranging contents according to the principles of tutorials, how-to guides, reference, and explanation.','A: Landing pages play a crucial role in the basic structure of documentation organized according to Diátaxis principles, serving as overview pages that provide context for the content within each section, such as describing what a tutorial has to offer or introducing the available how-to guides.'])

In [25]:
# format this for a dataset
def format_dataset(q, a):
    return {
        'conversations': [
            {"from:": "human", "value": q},
            {"from:": "gpt", "value": a},
        ]
    }

full_dataset= L(format_dataset(q, a) for q, a in zip(questions, pairs))

In [26]:
full_dataset[1]

{'conversations': [{'from:': 'human',
   'value': 'Q: What role do landing pages play in the basic structure of documentation organized according to Diátaxis principles?'},
  {'from:': 'gpt',
   'value': 'A: Landing pages play a crucial role in the basic structure of documentation organized according to Diátaxis principles, serving as overview pages that provide context for the content within each section, such as describing what a tutorial has to offer or introducing the available how-to guides.'}]}

In [27]:
# convert the dataset to a pandas dataframe, save the file as json
import pandas as pd
df = pd.DataFrame(full_dataset, columns=["conversations"])
df.to_json("diataxis.json", orient="records")

In [8]:
# load the dataset, sanity check...
dataset = Dataset.from_pandas(df)

In [9]:
dataset[0]]

SyntaxError: unmatched ']' (2128464850.py, line 1)